<a href="https://colab.research.google.com/github/odango314159/caTech/blob/main/LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install accelerate

In [ ]:
!pip install peft
!pip install datasets

In [53]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
from accelerate import Accelerator


model = AutoModelForCausalLM.from_pretrained("cyberagent/open-calm-small",low_cpu_mem_usage=False,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("cyberagent/open-calm-small")

In [36]:
import datasets

dolly_ja = datasets.load_dataset("kunishou/databricks-dolly-15k-ja")

In [37]:
dolly_ja['train'][0]

{'category': 'closed_qa',
 'index': '0',
 'instruction': 'ヴァージン・オーストラリア航空はいつから運航を開始したのですか？',
 'output': 'ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しました。',
 'input': 'ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。'}

In [38]:
dolly_ja = list(dolly_ja['train'])

In [39]:
PROMPT_DICT = {
    "prompt_input":(
        "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。"
        "要求を適切に満たす応答を書きなさい。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:{input}\n\n### 応答:"
    ),
    "prompt_no_input":(
        "以下は、タスクを説明する指示です。"
        "要求を適切に満たす応答を書きなさい。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    )
}

In [40]:
from transformers.models.deprecated.tapex.tokenization_tapex import json
import copy
from tqdm import tqdm
from torch.utils.data import Dataset

class InstructDataset(Dataset):
  def __init__(self,json_list,tokenizer,ignore_index=-100):
    ###tokenizerの定義
    self.tokenizer = tokenizer
    ###
    self.ignore_index = ignore_index
    self.json_list = json_list
    self.features = []

    for j in tqdm(json_list):
      if 'input' in j:
        ###取り出してきたjsonファイルに'input'キーが存在した時
        ###source_textをPROMPT_DICTの'prompt_input(文脈アリ)にして
        """

        (
        "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。"
        "要求を適切に満たす応答を書きなさい。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:{input}\n\n### 応答:"
    )


       instructionにjsonファイルのinstruction,inputにjsonファイルのinputを代入した値にして
       応答:までを問題文とする。
        """
        source_text = PROMPT_DICT['prompt_input'].format_map(j)
      else:
        source_text = PROMPT_DICT['prompt_no_input'].format_map(j)

      example_text = source_text + j['output'] + self.tokenizer.eos_token

      source_tokenized = self.tokenizer(
          source_text,
          padding = 'longest',
          truncation = True,
          max_length = 512,
          return_length = True,
          return_tensors = 'pt'
      )

      example_tokenized = self.tokenizer(
          example_text,
          padding = 'longest',
          truncation = True,
          max_length = 512,
          return_tensors = 'pt'
      )

      input_ids = example_tokenized['input_ids'][0]

      labels = copy.deepcopy(input_ids)

      source_len = source_tokenized['length'][0]

      labels[:source_len] = self.ignore_index

      self.features.append(
          {
              'input_ids':input_ids,
              'labels':labels
          }
      )
  def __len__(self):
    return len(self.features)
  def __getitem__(self,idx):
    return self.features[idx]

In [41]:
train_dataset = InstructDataset(dolly_ja,tokenizer)

100%|██████████| 15015/15015 [00:29<00:00, 508.53it/s]


In [42]:
from torch.nn.utils.rnn import pad_sequence

class InstructCollator():
  def __init__(self,tokenizer,ignore_index=-100):
    self.tokenizer = tokenizer
    self.ignore_index = -100

  def __call__(self,examples):
    input_batch = []
    label_batch = []
    for example in examples:
      input_batch.append(example['input_ids'])
      label_batch.append(example['labels'])

    input_ids = pad_sequence(
        input_batch,batch_first=True,padding_value=self.tokenizer.pad_token_id
    )

    labels = pad_sequence(
        label_batch,batch_first=True,padding_value=self.ignore_index
    )

    attention_mask = input_ids.ne(self.tokenizer.pad_token_id)

    return {
        'input_ids':input_ids,
        'labels':labels,
        'attention_mask':attention_mask
    }

In [43]:
from torch.utils.data import DataLoader

collator = InstructCollator(tokenizer)
loader = DataLoader(train_dataset,collate_fn=collator,batch_size=8,shuffle=True)

batch = next(iter(loader))
batch

{'input_ids': tensor([[24284,   245, 14946,  ...,     1,     1,     1],
         [24284,   245, 14946,  ...,     1,     1,     1],
         [24284,   245, 14946,  ...,     1,     1,     1],
         ...,
         [24284,   245, 14946,  ...,     1,     1,     1],
         [24284,   245, 14946,  ...,     1,     1,     1],
         [24284,   245, 14946,  ...,     1,     1,     1]]),
 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         ...,
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100]]),
 'attention_mask': tensor([[ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         [ True,  True,  True,  ..., False, False, False],
         ...,
         [ True,  True,  True,  ..., False, False, False],
     

In [44]:
import torch.nn as nn

for param in model.parameters():
  param.requires_grad = False
  if param.ndim ==1:
    param.data = param.data.to(torch.float32)

In [45]:
import torch.nn as nn

for param in model.parameters():
    param.requires_grad = False # モデルをフリーズ
    if param.ndim == 1:
        # 安定のためにレイヤーノルムをfp32にキャスト
        param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.embed_out = CastOutputToFloat(model.embed_out)

In [46]:
model.gpt_neox.layers[0].attention

GPTNeoXAttention(
  (rotary_emb): GPTNeoXRotaryEmbedding()
  (query_key_value): Linear(in_features=768, out_features=2304, bias=True)
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (attention_dropout): Dropout(p=0.0, inplace=False)
)

In [47]:
from peft import get_peft_model,LoraConfig,TaskType

In [54]:
lora_config = LoraConfig(
    r=8,
    lora_alpha = 32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    fan_in_fan_out = False,
    task_type = TaskType.CAUSAL_LM
)

model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 165,370,368 || trainable%: 0.1783342466771314


In [55]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
        output_dir='./output',
        save_total_limit=1,
        per_device_train_batch_size=8,
        num_train_epochs=1,
        remove_unused_columns=False,
        logging_steps=20,
        fp16=True,
        dataloader_num_workers=16,
        report_to="none",
)

In [56]:
trainer = Trainer(
        model=model,
        data_collator=collator,
        args=training_args,
        train_dataset=train_dataset,
    )

In [ ]:
model.config.use_cache = False
trainer.train()

In [58]:
model.save_pretrained("./output")